In [1]:
#dependencies
import re
from bs4 import BeautifulSoup 
from splinter import Browser
import requests
import pandas as pd
from selenium import webdriver 
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as BraveService
from webdriver_manager.core.utils import ChromeType
import json
import time
from tqdm.notebook import tqdm 

<h4> Start cells here if partial FIPS already available in {state}_fips_scraped file

In [2]:
fips_check_scraped = pd.read_csv("../../../data/state_data/geo/geo_fips/23/MD_fips_scraped.csv") 
fips_check_scraped  = fips_check_scraped.loc[:, ~fips_check_scraped.columns.str.contains('^Unnamed')] 
fips_check_scraped.head() 

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,4142717108,MD,"501 Fairmount Ave, Towson, MD",39.400438,-76.597013,2.400549e+14
1,4682157009,MD,"PO Box 2446 32 Enterprise Drive, OAKLAND, MD",39.406150,-79.355064,2.402300e+14
2,8298497108,MD,"1332 Londontown Blvd 200, Sykesville, MD",39.407222,-76.944486,2.401351e+14
3,8415647105,MD,"7750 Governor Ritchie Highway, Glen Burnie, MD",39.151658,-76.613994,2.400373e+14
4,9405257009,MD,"8184 Lark Brown Road Suite 201, ELKRIDGE, MD",39.189665,-76.794909,2.402761e+14


In [3]:
#Latitude, Longitude, not included
fips_check_scraped_notna = fips_check_scraped[fips_check_scraped["Lat"].notna()] # check to make sure no nulls in coords
fips_check_scraped_notna.head() 

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,4142717108,MD,"501 Fairmount Ave, Towson, MD",39.400438,-76.597013,2.400549e+14
1,4682157009,MD,"PO Box 2446 32 Enterprise Drive, OAKLAND, MD",39.406150,-79.355064,2.402300e+14
2,8298497108,MD,"1332 Londontown Blvd 200, Sykesville, MD",39.407222,-76.944486,2.401351e+14
3,8415647105,MD,"7750 Governor Ritchie Highway, Glen Burnie, MD",39.151658,-76.613994,2.400373e+14
4,9405257009,MD,"8184 Lark Brown Road Suite 201, ELKRIDGE, MD",39.189665,-76.794909,2.402761e+14


In [4]:
#FIPS not included, of Lat, Long available
fips_check_scraped_null = fips_check_scraped_notna[fips_check_scraped_notna["FIPS_z"].isnull()] # check to make sure no nulls in coords
fips_check_scraped_null #28093

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
455,5288307405,MD,"821 kennedy st nw, washington, MD",38.956476,-77.025084,NaN
1724,6109407204,MD,"209 Perry Parkway, SUITE 8, GAITHERSBURG, MD",39.143533,-77.209355,NaN
2028,4365897102,MD,"1000 Harborview Ct., SUDLERSVILLE, MD",41.603221,-73.087749,NaN
2068,9453537004,MD,"201 Le Phillip Ct NE, CHEVY CHASE, MD",41.603221,-73.087749,NaN
4992,7847078400,MD,"6751 Columbia Gateway Dr STE 300, COLUMBIA, MD",39.184250,-76.801541,NaN
5228,2085197209,MD,"9111 Edmonston Road, Suite 303, Greenbelt, MD",38.902914,-77.033066,NaN
6061,9654698009,MD,"621 Stemmers Run Rd Ste B, Essex, MD",39.327441,-76.467014,NaN
9338,4458978805,MD,"7404 Executive Pl Ste 400, Lanham, MD",38.992494,-76.825910,NaN
9750,1713177200,MD,"67 W. Baltimore St. Ste 101, HAGERSTOWN, MD",39.639347,-77.725094,NaN
10941,5522567107,MD,"5594 ASHLAND COMMUNITY SQ, EDGEWATER, MD",38.613268,-77.373759,NaN


<h4>FIPS Script

In [5]:
#if FIPS_z file 
lat = fips_check_scraped_null["Lat"].astype(float).astype(str).to_list()
long = fips_check_scraped_null["Long"].astype(float).astype(str).to_list()

In [6]:
#before running script, check for any NaN within Lat, Long fields. 
def sleep():
    time.sleep(1)
option = webdriver.ChromeOptions()
option.binary_location = "/Applications/Brave Browser.app/Contents/MacOS/Brave Browser"
driver = Browser('chrome', options=option, headless=False)    
url = 'https://geocoding.geo.census.gov/geocoder/geographies/coordinates?form'
driver.visit(url)

not_found = []
geo_id = []

i=0
for i in tqdm(range(len(lat))):
    #while i < len(lat):
        driver.find_by_id('geoCoordX').clear()
        driver.find_by_id('geoCoordY').clear()
        driver.find_by_id('geoCoordX').fill(long[i])
        driver.find_by_id('geoCoordY').fill(lat[i])
        driver.find_by_id('geoCoordsSubmit').click()
        #sleep()
        #html = driver.html
        soup = BeautifulSoup(driver.html, 'html.parser')


        results = soup.find_all('div', id='geoCoordsJsonOutput')
        for result in results:
            result_text = result.text

            result_digits = re.findall('\d+', result_text)
            startswith = [s for s in result_digits if s.startswith('24')] #MD
            fips_elements =[element for element in startswith if len(element)>=15]
            
            try:
                #print(f"{i},{lat[i]},{long[i]} is {fips_elements}")
                geo_id.append(fips_elements)
            except (KeyError, IndexError):
                #print(f"{i},{lat[i]},{long[i]} Not Found")
                not_found.append(i)
                geo_id.append(None)  
            except (TimeoutException):
                driver.navigate.refresh()
    
        #sleep()
        i=i+1

  0%|          | 0/20 [00:00<?, ?it/s]

coordinates wtih more than one return:
385,37.514070000000004,-77.35472 is ['514070000000004', '510872015041003'] 
1178,37.516454700000004,-77.8100575 is ['516454700000004', '511455001022014'] 
2660,37.519462299999994,-77.340384 is ['519462299999994', '510872014061012']
second option reference FIPS code, first option references LatCoord, update fields in DataFrame

https://iqss.github.io/dss-webscrape/filling-in-web-forms.html

In [7]:
geo_id_list =geo_id 

Length Check

In [8]:
print(len(geo_id_list))

20


Matching FIPS to DataFrame

In [9]:
fips_fill = fips_check_scraped_null

In [10]:
fips_fill["FIPS_z"] = geo_id_list
fips_fill

/var/folders/vy/qdp8xcfj4s3dd5c6d1xp_y3w0000gn/T/ipykernel_12294/597263667.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fips_fill["FIPS_z"] = geo_id_list


,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
455,5288307405,MD,"821 kennedy st nw, washington, MD",38.956476,-77.025084,[]
1724,6109407204,MD,"209 Perry Parkway, SUITE 8, GAITHERSBURG, MD",39.143533,-77.209355,[240317007242000]
2028,4365897102,MD,"1000 Harborview Ct., SUDLERSVILLE, MD",41.603221,-73.087749,[]
2068,9453537004,MD,"201 Le Phillip Ct NE, CHEVY CHASE, MD",41.603221,-73.087749,[]
4992,7847078400,MD,"6751 Columbia Gateway Dr STE 300, COLUMBIA, MD",39.184250,-76.801541,[240276067081013]
5228,2085197209,MD,"9111 Edmonston Road, Suite 303, Greenbelt, MD",38.902914,-77.033066,[]
6061,9654698009,MD,"621 Stemmers Run Rd Ste B, Essex, MD",39.327441,-76.467014,[240054503001000]
9338,4458978805,MD,"7404 Executive Pl Ste 400, Lanham, MD",38.992494,-76.825910,[240338004111028]
9750,1713177200,MD,"67 W. Baltimore St. Ste 101, HAGERSTOWN, MD",39.639347,-77.725094,[240430008001002]
10941,5522567107,MD,"5594 ASHLAND COMMUNITY SQ, EDGEWATER, MD",38.613268,-77.373759,[]


Remove brackets

In [11]:
fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]
fips_fill

/var/folders/vy/qdp8xcfj4s3dd5c6d1xp_y3w0000gn/T/ipykernel_12294/1219885402.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]


,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
455,5288307405,MD,"821 kennedy st nw, washington, MD",38.956476,-77.025084,NaN
1724,6109407204,MD,"209 Perry Parkway, SUITE 8, GAITHERSBURG, MD",39.143533,-77.209355,240317007242000
2028,4365897102,MD,"1000 Harborview Ct., SUDLERSVILLE, MD",41.603221,-73.087749,NaN
2068,9453537004,MD,"201 Le Phillip Ct NE, CHEVY CHASE, MD",41.603221,-73.087749,NaN
4992,7847078400,MD,"6751 Columbia Gateway Dr STE 300, COLUMBIA, MD",39.184250,-76.801541,240276067081013
5228,2085197209,MD,"9111 Edmonston Road, Suite 303, Greenbelt, MD",38.902914,-77.033066,NaN
6061,9654698009,MD,"621 Stemmers Run Rd Ste B, Essex, MD",39.327441,-76.467014,240054503001000
9338,4458978805,MD,"7404 Executive Pl Ste 400, Lanham, MD",38.992494,-76.825910,240338004111028
9750,1713177200,MD,"67 W. Baltimore St. Ste 101, HAGERSTOWN, MD",39.639347,-77.725094,240430008001002
10941,5522567107,MD,"5594 ASHLAND COMMUNITY SQ, EDGEWATER, MD",38.613268,-77.373759,NaN


In [12]:
fips_fill.loc[fips_fill["FIPS_z"].isnull()] #check #13

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
455,5288307405,MD,"821 kennedy st nw, washington, MD",38.956476,-77.025084,NaN
2028,4365897102,MD,"1000 Harborview Ct., SUDLERSVILLE, MD",41.603221,-73.087749,NaN
2068,9453537004,MD,"201 Le Phillip Ct NE, CHEVY CHASE, MD",41.603221,-73.087749,NaN
5228,2085197209,MD,"9111 Edmonston Road, Suite 303, Greenbelt, MD",38.902914,-77.033066,NaN
10941,5522567107,MD,"5594 ASHLAND COMMUNITY SQ, EDGEWATER, MD",38.613268,-77.373759,NaN
13009,3652408307,MD,"12614 Nicklaus Ct, Cumberland, MD",41.603221,-73.087749,NaN
14538,5893948904,MD,"4075 Wilson Blvd STE 800, Arlington, MD",38.880246,-77.108616,NaN
14732,6542178400,MD,"2807 Bel Ct, Manchester, MD",41.603221,-73.087749,NaN
14948,6862267700,MD,"2807 BEL CT, MANCHESTER, MD",41.603221,-73.087749,NaN
15573,5293057200,MD,"19 WISCONSIN AVE, CHEVY CHASE, MD",38.959033,-77.084842,NaN


Parse for Merge

In [13]:
cols= ["LoanNumber", "FIPS_z"]
fips_fill_parsed = fips_fill[cols]
fips_fill_parsed

,LoanNumber,FIPS_z
455,5288307405,NaN
1724,6109407204,240317007242000
2028,4365897102,NaN
2068,9453537004,NaN
4992,7847078400,240276067081013
5228,2085197209,NaN
6061,9654698009,240054503001000
9338,4458978805,240338004111028
9750,1713177200,240430008001002
10941,5522567107,NaN


Read in base data

In [14]:

#if from FIPS file
fips_check = pd.read_csv("../../../data/state_data/geo/geo_fips/23/MD_fips_scraped.csv") 
fips_check.head()

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,423723,4142717108,MD,"501 Fairmount Ave, Towson, MD",39.400438,-76.597013,2.400549e+14
1,423724,4682157009,MD,"PO Box 2446 32 Enterprise Drive, OAKLAND, MD",39.406150,-79.355064,2.402300e+14
2,423725,8298497108,MD,"1332 Londontown Blvd 200, Sykesville, MD",39.407222,-76.944486,2.401351e+14
3,423726,8415647105,MD,"7750 Governor Ritchie Highway, Glen Burnie, MD",39.151658,-76.613994,2.400373e+14
4,423727,9405257009,MD,"8184 Lark Brown Road Suite 201, ELKRIDGE, MD",39.189665,-76.794909,2.402761e+14


Merge with MainFrame

In [15]:
fips_merge = fips_check.merge(fips_fill_parsed, on="LoanNumber", how="left")
fips_merge

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z_x,FIPS_z_y
0,423723,4142717108,MD,"501 Fairmount Ave, Towson, MD",39.400438,-76.597013,2.400549e+14,NaN
1,423724,4682157009,MD,"PO Box 2446 32 Enterprise Drive, OAKLAND, MD",39.406150,-79.355064,2.402300e+14,NaN
2,423725,8298497108,MD,"1332 Londontown Blvd 200, Sykesville, MD",39.407222,-76.944486,2.401351e+14,NaN
3,423726,8415647105,MD,"7750 Governor Ritchie Highway, Glen Burnie, MD",39.151658,-76.613994,2.400373e+14,NaN
4,423727,9405257009,MD,"8184 Lark Brown Road Suite 201, ELKRIDGE, MD",39.189665,-76.794909,2.402761e+14,NaN
...,...,...,...,...,...,...,...,...
18950,442673,8964248309,MD,"7401 Woodmont Ave, Bethesda, MD",38.983328,-77.094790,2.403170e+14,NaN
18951,442674,9164838305,MD,"411 Aviation Way Ste 220, Frederick, MD",39.418681,-77.380239,2.402177e+14,NaN
18952,442675,9216407410,MD,"6537 BALTIMORE NATIONAL PIKE, CATONSVILLE, MD",39.285835,-76.765583,2.400540e+14,NaN
18953,442676,9305188501,MD,"32F Queen Anne Way, Chester, MD",38.984626,-76.280169,2.403581e+14,NaN


<b>Fill FIPS Columns - if FIPS file

In [16]:
fips_merge["FIPS_z"] = fips_merge["FIPS_z_y"].fillna(fips_merge["FIPS_z_x"])
fips_merge.head()

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z_x,FIPS_z_y,FIPS_z
0,423723,4142717108,MD,"501 Fairmount Ave, Towson, MD",39.400438,-76.597013,2.400549e+14,NaN,240054909002002.0
1,423724,4682157009,MD,"PO Box 2446 32 Enterprise Drive, OAKLAND, MD",39.406150,-79.355064,2.402300e+14,NaN,240230007004094.0
2,423725,8298497108,MD,"1332 Londontown Blvd 200, Sykesville, MD",39.407222,-76.944486,2.401351e+14,NaN,240135051021008.0
3,423726,8415647105,MD,"7750 Governor Ritchie Highway, Glen Burnie, MD",39.151658,-76.613994,2.400373e+14,NaN,240037304041001.0
4,423727,9405257009,MD,"8184 Lark Brown Road Suite 201, ELKRIDGE, MD",39.189665,-76.794909,2.402761e+14,NaN,240276067061004.0


Drop Extra FIPS Columns

In [17]:
fips_merge_drop = fips_merge.drop(["FIPS_z_x", "FIPS_z_y"], axis=1)
fips_merge_drop = fips_merge_drop.drop(["Unnamed: 0"], axis=1)
fips_merge_drop

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,4142717108,MD,"501 Fairmount Ave, Towson, MD",39.400438,-76.597013,240054909002002.0
1,4682157009,MD,"PO Box 2446 32 Enterprise Drive, OAKLAND, MD",39.406150,-79.355064,240230007004094.0
2,8298497108,MD,"1332 Londontown Blvd 200, Sykesville, MD",39.407222,-76.944486,240135051021008.0
3,8415647105,MD,"7750 Governor Ritchie Highway, Glen Burnie, MD",39.151658,-76.613994,240037304041001.0
4,9405257009,MD,"8184 Lark Brown Road Suite 201, ELKRIDGE, MD",39.189665,-76.794909,240276067061004.0
...,...,...,...,...,...,...
18950,8964248309,MD,"7401 Woodmont Ave, Bethesda, MD",38.983328,-77.094790,240317048041002.0
18951,9164838305,MD,"411 Aviation Way Ste 220, Frederick, MD",39.418681,-77.380239,240217722002005.0
18952,9216407410,MD,"6537 BALTIMORE NATIONAL PIKE, CATONSVILLE, MD",39.285835,-76.765583,240054015031001.0
18953,9305188501,MD,"32F Queen Anne Way, Chester, MD",38.984626,-76.280169,240358108003004.0


In [18]:
fips_merge_drop.loc[fips_merge_drop["FIPS_z"].isnull()] #13

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
455,5288307405,MD,"821 kennedy st nw, washington, MD",38.956476,-77.025084,NaN
2028,4365897102,MD,"1000 Harborview Ct., SUDLERSVILLE, MD",41.603221,-73.087749,NaN
2068,9453537004,MD,"201 Le Phillip Ct NE, CHEVY CHASE, MD",41.603221,-73.087749,NaN
5228,2085197209,MD,"9111 Edmonston Road, Suite 303, Greenbelt, MD",38.902914,-77.033066,NaN
10941,5522567107,MD,"5594 ASHLAND COMMUNITY SQ, EDGEWATER, MD",38.613268,-77.373759,NaN
13009,3652408307,MD,"12614 Nicklaus Ct, Cumberland, MD",41.603221,-73.087749,NaN
14538,5893948904,MD,"4075 Wilson Blvd STE 800, Arlington, MD",38.880246,-77.108616,NaN
14732,6542178400,MD,"2807 Bel Ct, Manchester, MD",41.603221,-73.087749,NaN
14948,6862267700,MD,"2807 BEL CT, MANCHESTER, MD",41.603221,-73.087749,NaN
15573,5293057200,MD,"19 WISCONSIN AVE, CHEVY CHASE, MD",38.959033,-77.084842,NaN


Convert to File

In [19]:
fips_MD = fips_merge_drop

In [20]:
fips_MD.to_csv("../../../data/state_data/geo/geo_fips/23/MD_fips_scraped.csv") 